# TD SIEM : Suricata, Filebeat, Elasticsearch et Kibana

## Objectifs du TD

Ce TD vous permettra de :
- Configurer Suricata (système de détection d'intrusion)
- Utiliser Filebeat pour collecter et envoyer les logs vers Elasticsearch
- Explorer et analyser les données dans Elasticsearch
- Créer des visualisations dans Kibana
- Configurer des alertes pour la détection de menaces
- Utiliser la détection d'anomalies avec Machine Learning

## Architecture SIEM

```
Suricata → Filebeat → Elasticsearch → Kibana
   (IDS)    (Collecteur)   (Stockage)   (Visualisation/Alerting)
```

## Prérequis

### 1. Installation des dépendances

Assurez-vous d'avoir installé les dépendances du projet :
```bash
uv sync
```

### 2. Démarrage de la stack complète SIEM

Le fichier `docker-compose-siem.yml` inclut toute la stack nécessaire pour ce TD :
- Cluster Elasticsearch (3 nœuds)
- Kibana
- Suricata (IDS/IPS)
- Filebeat (collecteur de logs)

**Démarrage de la stack complète :**
```bash
docker-compose -f docker-compose-siem.yml up -d
```

**Vérification :**
- Elasticsearch : `https://localhost:9200` (ou le port configuré dans `.env`)
- Kibana : `http://localhost:5601` (ou le port configuré dans `.env`)

**Note :** Si vous utilisez un cluster avec sécurité activée, vous devrez utiliser les identifiants définis dans le fichier `.env` (ELASTIC_PASSWORD).

### 3. Préparation des répertoires

Avant de démarrer la stack, créez les répertoires nécessaires pour Suricata :

```bash
mkdir -p suricata/{logs,config,rules}
mkdir -p suricata_logs
```

**Note :** Tous les services (Elasticsearch, Kibana, Suricata, Filebeat) sont déjà configurés dans le fichier `docker-compose.yml` et seront démarrés ensemble.

### 4. Vérification de la stack

La cellule suivante vérifie que tous les services sont correctement démarrés.

In [1]:
# Vérification de la stack Docker
import subprocess
import os

def check_docker_container(container_name):
    """Vérifie si un conteneur Docker est en cours d'exécution"""
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', f'name={container_name}', '--format', '{{.Names}}'],
            capture_output=True,
            text=True,
            timeout=5
        )
        return container_name in result.stdout
    except Exception:
        return False

def check_docker_compose():
    """Vérifie si docker-compose est disponible"""
    try:
        subprocess.run(['docker', 'compose', 'version'], capture_output=True, check=True, timeout=5)
        return True
    except:
        try:
            subprocess.run(['docker-compose', 'version'], capture_output=True, check=True, timeout=5)
            return True
        except:
            return False

# Vérifier Docker
try:
    subprocess.run(['docker', '--version'], capture_output=True, check=True, timeout=5)
    print("✅ Docker est installé")
except (FileNotFoundError, subprocess.CalledProcessError):
    print("❌ Docker n'est pas installé")
    print("   Installez Docker: https://docs.docker.com/get-docker/")
    exit(1)

# Vérifier docker-compose
if check_docker_compose():
    print("✅ Docker Compose est disponible")
else:
    print("❌ Docker Compose n'est pas disponible")

# Vérifier les conteneurs
services = ['es01', 'es02', 'es03', 'kibana', 'suricata', 'filebeat']
print("\n📦 Statut des services:")
for service in services:
    if check_docker_container(service):
        print(f"   ✅ {service}: en cours d'exécution")
    else:
        print(f"   ⚠️  {service}: arrêté")

print("\n📝 Pour démarrer la stack complète:")
print("   docker-compose up -d")
print("\n📝 Pour arrêter:")
print("   docker-compose down")

✅ Docker est installé
✅ Docker Compose est disponible

📦 Statut des services:
   ✅ es01: en cours d'exécution


   ✅ es02: en cours d'exécution
   ✅ es03: en cours d'exécution
   ✅ kibana: en cours d'exécution
   ✅ suricata: en cours d'exécution


   ⚠️  filebeat: arrêté

📝 Pour démarrer la stack complète:
   docker-compose up -d

📝 Pour arrêter:
   docker-compose down


In [2]:
# Import des librairies nécessaires
from elasticsearch import Elasticsearch
from pprint import pprint
import json
import time
from datetime import datetime, timedelta

# Configuration de la connexion Elasticsearch
# Adaptez selon votre configuration (avec ou sans sécurité)
ES_HOST = "https://localhost:9200"
ES_USER = "elastic"  # Modifiez si nécessaire
ES_PASSWORD = "changeme"  # Définissez votre mot de passe si sécurité activée

# Connexion au cluster Elasticsearch
if ES_PASSWORD:
    es = Elasticsearch(
        [ES_HOST],
        basic_auth=(ES_USER, ES_PASSWORD),
        verify_certs=False  # Désactiver la vérification SSL pour le développement
    )
else:
    # Pour un cluster sans sécurité (développement uniquement)
    es = Elasticsearch("http://localhost:9200")

# Vérification de la connexion
try:
    health = es.cluster.health()
    print("✅ Connexion établie avec Elasticsearch")
    print(f"   Statut du cluster: {health['status']}")
    print(f"   Nombre de nœuds: {health['number_of_nodes']}")
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")
    print("   Vérifiez que votre cluster Elasticsearch est démarré")

✅ Connexion établie avec Elasticsearch
   Statut du cluster: green
   Nombre de nœuds: 3


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/elasticsearch/_sync/client/__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Partie 1 : Configuration de Suricata

### 1.1 Vérification de l'installation de Suricata

In [3]:
# Création des répertoires nécessaires pour Suricata
import os

# Créer les répertoires si nécessaire
directories = [
    './suricata/logs',
    './suricata/config',
    './suricata/rules',
    './suricata_logs'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Répertoire créé/vérifié: {directory}")

print("\n📝 Les répertoires sont prêts pour Suricata")

✅ Répertoire créé/vérifié: ./suricata/logs
✅ Répertoire créé/vérifié: ./suricata/config
✅ Répertoire créé/vérifié: ./suricata/rules
✅ Répertoire créé/vérifié: ./suricata_logs

📝 Les répertoires sont prêts pour Suricata


### 1.2 Configuration de Suricata pour générer des logs JSON

Suricata génère des logs dans différents formats. Pour ce TD, nous allons configurer Suricata pour générer des logs au format JSON (EVE logs) qui sont plus faciles à parser.

**Avec Docker :** La configuration se trouve dans `./suricata/config/suricata.yaml` (monté dans le conteneur)

**Fichier de configuration :** `./suricata/config/suricata.yaml` (dans le répertoire du projet)

**Configuration recommandée :**
```yaml
# Activer les logs EVE (JSON)
- eve-log:
    enabled: yes
    filetype: regular
    filename: eve.json
    types:
      - alert
      - http
      - dns
      - tls
      - files
      - ssh
      - stats
      - flow
      - netflow
    # Format JSON pour faciliter le parsing
    json: yes
```

**Note :** 
- Pour ce TD, nous allons créer des logs de test qui seront utilisés même si Suricata Docker n'est pas configuré
- Les logs seront générés dans `./suricata/logs/eve.json` (monté dans le conteneur)
- Pour une utilisation réelle, vous devrez configurer l'interface réseau dans `suricata.yaml`

In [4]:
# Création de logs Suricata de test (format EVE JSON)
# Ces logs simulent des alertes Suricata typiques

import os
import builtins

# Créer un répertoire pour les logs si nécessaire (utiliser un répertoire accessible)
log_dir = "./suricata_logs"
try:
    os.makedirs(log_dir, exist_ok=True, mode=0o755)
    # Essayer de créer un fichier test pour vérifier les permissions
    test_file = os.path.join(log_dir, ".test_write")
    with builtins.open(test_file, 'w') as f:
        f.write("test")
    os.remove(test_file)
except (PermissionError, OSError) as e:
    # Si on ne peut pas écrire dans suricata_logs, utiliser un autre répertoire
    log_dir = "./suricata_logs_local"
    os.makedirs(log_dir, exist_ok=True, mode=0o755)
    print(f"⚠️  Utilisation du répertoire alternatif: {log_dir}")

# Exemple de logs Suricata au format EVE JSON
sample_logs = [
    {
        "timestamp": datetime.now().isoformat(),
        "event_type": "alert",
        "src_ip": "192.168.1.100",
        "src_port": 54321,
        "dest_ip": "10.0.0.50",
        "dest_port": 80,
        "proto": "TCP",
        "alert": {
            "action": "allowed",
            "gid": 1,
            "signature_id": 2012896,
            "rev": 1,
            "signature": "ET POLICY Suspicious inbound to mySQL port 3306",
            "category": "Potentially Bad Traffic",
            "severity": 2
        },
        "flow_id": 1234567890,
        "in_iface": "eth0"
    },
    {
        "timestamp": (datetime.now() - timedelta(minutes=5)).isoformat(),
        "event_type": "alert",
        "src_ip": "203.0.113.45",
        "src_port": 44321,
        "dest_ip": "10.0.0.50",
        "dest_port": 22,
        "proto": "TCP",
        "alert": {
            "action": "allowed",
            "gid": 1,
            "signature_id": 2012897,
            "rev": 1,
            "signature": "ET SCAN Potential SSH Scan",
            "category": "Attempted Information Leak",
            "severity": 2
        },
        "flow_id": 1234567891,
        "in_iface": "eth0"
    },
    {
        "timestamp": (datetime.now() - timedelta(minutes=10)).isoformat(),
        "event_type": "http",
        "src_ip": "192.168.1.100",
        "src_port": 54322,
        "dest_ip": "10.0.0.50",
        "dest_port": 80,
        "proto": "TCP",
        "http": {
            "hostname": "example.com",
            "url": "/admin/login.php",
            "http_user_agent": "Mozilla/5.0",
            "http_method": "POST",
            "status": 200
        },
        "flow_id": 1234567892
    },
    {
        "timestamp": (datetime.now() - timedelta(minutes=15)).isoformat(),
        "event_type": "dns",
        "src_ip": "192.168.1.100",
        "src_port": 54323,
        "dest_ip": "8.8.8.8",
        "dest_port": 53,
        "proto": "UDP",
        "dns": {
            "type": "query",
            "id": 12345,
            "rrname": "malicious-domain.com",
            "rrtype": "A"
        },
        "flow_id": 1234567893
    },
    {
        "timestamp": (datetime.now() - timedelta(minutes=20)).isoformat(),
        "event_type": "alert",
        "src_ip": "198.51.100.10",
        "src_port": 44324,
        "dest_ip": "10.0.0.50",
        "dest_port": 443,
        "proto": "TCP",
        "alert": {
            "action": "allowed",
            "gid": 1,
            "signature_id": 2012898,
            "rev": 1,
            "signature": "ET MALWARE Known Malware IP",
            "category": "A Network Trojan was detected",
            "severity": 1
        },
        "flow_id": 1234567894,
        "in_iface": "eth0"
    }
]

# Écrire les logs dans un fichier (utiliser builtins.open pour éviter le problème IPython)
log_file = os.path.join(log_dir, "eve.json")
with builtins.open(log_file, 'w') as f:
    for log in sample_logs:
        f.write(json.dumps(log) + '\n')

print(f"✅ Logs de test créés dans {log_file}")
print(f"   Nombre de logs: {len(sample_logs)}")

⚠️  Utilisation du répertoire alternatif: ./suricata_logs_local
✅ Logs de test créés dans ./suricata_logs_local/eve.json
   Nombre de logs: 5


In [5]:
# Note: Les permissions ont été gérées dans la cellule précédente
# Si vous avez besoin de modifier les permissions, utilisez:
# chmod -R 755 ./suricata_logs/
# (sans sudo si vous êtes propriétaire du répertoire)

In [6]:
# Création de la configuration Filebeat pour Suricata
# Cette configuration utilise HTTPS et l'authentification pour se connecter à Elasticsearch
filebeat_config = {
    "filebeat.inputs": [
        {
            "type": "log",
            "enabled": True,
            "paths": [
                "/var/log/suricata/eve.json"
            ],
            "json.keys_under_root": True,
            "json.add_error_key": True,
            "json.message_key": "message",
            "fields": {
                "log_type": "suricata"
            },
            "fields_under_root": False
        }
    ],
    "output.elasticsearch": {
        "hosts": ["https://es01:9200"],
        "index": "suricata-%{+yyyy.MM.dd}",
        "username": "elastic",
        "password": "${ELASTICSEARCH_PASSWORD}",
        "ssl": {
            "certificate_authorities": ["/usr/share/filebeat/config/certs/ca/ca.crt"],
            "verification_mode": "certificate"
        }
    },
    "setup.template.name": "suricata",
    "setup.template.pattern": "suricata-*",
    "setup.template.settings": {
        "index.number_of_shards": 1,
        "index.codec": "best_compression"
    },
    "processors": [
        {
            "timestamp": {
                "field": "timestamp",
                "layouts": [
                    "2006-01-02T15:04:05.000000-0700",
                    "2006-01-02T15:04:05.000000Z",
                    "2006-01-02T15:04:05Z"
                ],
                "test": [
                    "2024-01-15T10:30:45.123456Z"
                ]
            }
        }
    ]
}

# Sauvegarder la configuration
import builtins
config_file = "filebeat_suricata.yml"
# Vérifier si c'est un répertoire et le supprimer si nécessaire
if os.path.exists(config_file) and os.path.isdir(config_file):
    import shutil
    shutil.rmtree(config_file)
    print(f"⚠️  Suppression du répertoire {config_file} (sera remplacé par un fichier)")
with builtins.open(config_file, 'w') as f:
    import yaml
    try:
        yaml.dump(filebeat_config, f, default_flow_style=False, sort_keys=False, allow_unicode=True)
    except ImportError:
        # Si PyYAML n'est pas installé, sauvegarder en JSON
        json.dump(filebeat_config, f, indent=2)
        print("⚠️  PyYAML non installé, configuration sauvegardée en JSON")
        print("   Pour installer PyYAML: pip install pyyaml")

print(f"✅ Configuration Filebeat créée: {config_file}")
print("\n📝 Note: Cette configuration sera utilisée automatiquement par le service Filebeat")
print("   dans docker-compose.yml. Le mot de passe sera injecté via la variable")
print("   d'environnement ELASTICSEARCH_PASSWORD.")

⚠️  Suppression du répertoire filebeat_suricata.yml (sera remplacé par un fichier)
✅ Configuration Filebeat créée: filebeat_suricata.yml

📝 Note: Cette configuration sera utilisée automatiquement par le service Filebeat
   dans docker-compose.yml. Le mot de passe sera injecté via la variable
   d'environnement ELASTICSEARCH_PASSWORD.


### 2.2 Indexation manuelle des logs dans Elasticsearch

Pour ce TD, nous allons indexer directement les logs dans Elasticsearch via l'API Python, ce qui évite d'avoir à installer et configurer Filebeat.

In [7]:
# Indexation des logs Suricata dans Elasticsearch
import os

# Lire les logs depuis le fichier (chercher dans les deux répertoires possibles)
log_file = "./suricata_logs/eve.json"
if not os.path.exists(log_file):
    log_file = "./suricata_logs_local/eve.json"
index_name = "suricata-logs"

# Créer un index avec mapping approprié pour les logs Suricata
mapping = {
    "mappings": {
        "properties": {
            "timestamp": {"type": "date"},
            "event_type": {"type": "keyword"},
            "src_ip": {"type": "ip"},
            "src_port": {"type": "integer"},
            "dest_ip": {"type": "ip"},
            "dest_port": {"type": "integer"},
            "proto": {"type": "keyword"},
            "alert": {
                "properties": {
                    "action": {"type": "keyword"},
                    "gid": {"type": "integer"},
                    "signature_id": {"type": "integer"},
                    "signature": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
                    "category": {"type": "keyword"},
                    "severity": {"type": "integer"}
                }
            },
            "http": {
                "properties": {
                    "hostname": {"type": "keyword"},
                    "url": {"type": "text"},
                    "http_method": {"type": "keyword"},
                    "status": {"type": "integer"}
                }
            },
            "dns": {
                "properties": {
                    "type": {"type": "keyword"},
                    "rrname": {"type": "keyword"},
                    "rrtype": {"type": "keyword"}
                }
            },
            "flow_id": {"type": "long"},
            "in_iface": {"type": "keyword"}
        }
    }
}

# Supprimer l'index s'il existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"🗑️  Index '{index_name}' supprimé")

# Créer l'index avec le mapping
es.indices.create(index=index_name, body=mapping)
print(f"✅ Index '{index_name}' créé avec mapping")

# Indexer les logs
if os.path.exists(log_file):
    with open(log_file, 'r') as f:
        logs = [json.loads(line) for line in f if line.strip()]
    
    # Indexer chaque log
    for i, log in enumerate(logs):
        # Convertir le timestamp en format date si nécessaire
        if 'timestamp' in log:
            try:
                # Essayer de parser le timestamp
                log['@timestamp'] = log['timestamp']
            except:
                pass
        
        es.index(index=index_name, document=log, id=i+1)
    
    # Rafraîchir l'index pour que les données soient disponibles
    es.indices.refresh(index=index_name)
    
    print(f"✅ {len(logs)} logs indexés dans '{index_name}'")
    
    # Vérifier le nombre de documents
    count = es.count(index=index_name)
    print(f"   Nombre total de documents: {count['count']}")
else:
    print(f"⚠️  Fichier {log_file} non trouvé")

/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Index 'suricata-logs' créé avec mapping


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanc

/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanc

✅ 5 logs indexés dans 'suricata-logs'
   Nombre total de documents: 5


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Partie 3 : Exploration des données dans Elasticsearch

### 3.1 Recherche de base dans les logs Suricata

In [8]:
# Recherche de tous les logs
query_all = {
    "match_all": {}
}

result = es.search(index=index_name, query=query_all, size=10)
print(f"📊 Total de documents: {result['hits']['total']['value']}")
print(f"   Documents retournés: {len(result['hits']['hits'])}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    print(f"📄 Document {hit['_id']}:")
    print(f"   Type: {source.get('event_type', 'N/A')}")
    if 'alert' in source:
        print(f"   Alerte: {source['alert'].get('signature', 'N/A')}")
    print(f"   Source IP: {source.get('src_ip', 'N/A')} → Dest IP: {source.get('dest_ip', 'N/A')}")
    print()

📊 Total de documents: 5
   Documents retournés: 5

📄 Document 1:
   Type: alert
   Alerte: ET POLICY Suspicious inbound to mySQL port 3306
   Source IP: 192.168.1.100 → Dest IP: 10.0.0.50

📄 Document 2:
   Type: alert
   Alerte: ET SCAN Potential SSH Scan
   Source IP: 203.0.113.45 → Dest IP: 10.0.0.50

📄 Document 3:
   Type: http
   Source IP: 192.168.1.100 → Dest IP: 10.0.0.50

📄 Document 4:
   Type: dns
   Source IP: 192.168.1.100 → Dest IP: 8.8.8.8

📄 Document 5:
   Type: alert
   Alerte: ET MALWARE Known Malware IP
   Source IP: 198.51.100.10 → Dest IP: 10.0.0.50



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.2 Recherche d'alertes spécifiques

In [9]:
# Rechercher toutes les alertes (event_type = "alert")
query_alerts = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}}
        ]
    }
}

result = es.search(index=index_name, query=query_alerts, size=100)
print(f"🚨 Nombre d'alertes trouvées: {result['hits']['total']['value']}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    alert = source.get('alert', {})
    print(f"⚠️  Alerte ID: {alert.get('signature_id', 'N/A')}")
    print(f"   Signature: {alert.get('signature', 'N/A')}")
    print(f"   Catégorie: {alert.get('category', 'N/A')}")
    print(f"   Sévérité: {alert.get('severity', 'N/A')}")
    print(f"   Source: {source.get('src_ip', 'N/A')}:{source.get('src_port', 'N/A')}")
    print(f"   Destination: {source.get('dest_ip', 'N/A')}:{source.get('dest_port', 'N/A')}")
    print()

🚨 Nombre d'alertes trouvées: 3

⚠️  Alerte ID: 2012896
   Signature: ET POLICY Suspicious inbound to mySQL port 3306
   Catégorie: Potentially Bad Traffic
   Sévérité: 2
   Source: 192.168.1.100:54321
   Destination: 10.0.0.50:80

⚠️  Alerte ID: 2012897
   Signature: ET SCAN Potential SSH Scan
   Catégorie: Attempted Information Leak
   Sévérité: 2
   Source: 203.0.113.45:44321
   Destination: 10.0.0.50:22

⚠️  Alerte ID: 2012898
   Signature: ET MALWARE Known Malware IP
   Catégorie: A Network Trojan was detected
   Sévérité: 1
   Source: 198.51.100.10:44324
   Destination: 10.0.0.50:443



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.3 Recherche d'alertes de haute sévérité

In [10]:
# Rechercher les alertes de haute sévérité (severity <= 1)
query_high_severity = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}},
            {"range": {"alert.severity": {"lte": 1}}}
        ]
    }
}

result = es.search(index=index_name, query=query_high_severity, size=100)
print(f"🔴 Alertes de haute sévérité (≤1): {result['hits']['total']['value']}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    alert = source.get('alert', {})
    print(f"🔴 CRITIQUE - Sévérité {alert.get('severity', 'N/A')}")
    print(f"   {alert.get('signature', 'N/A')}")
    print(f"   Source IP suspecte: {source.get('src_ip', 'N/A')}")
    print()

🔴 Alertes de haute sévérité (≤1): 1

🔴 CRITIQUE - Sévérité 1
   ET MALWARE Known Malware IP
   Source IP suspecte: 198.51.100.10



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.4 Agrégations : Statistiques sur les alertes

In [11]:
# Agrégations pour analyser les données
aggs_query = {
    "size": 0,
    "aggs": {
        "alertes_par_categorie": {
            "terms": {
                "field": "alert.category",
                "size": 10
            }
        },
        "alertes_par_severite": {
            "terms": {
                "field": "alert.severity",
                "size": 5
            }
        },
        "top_source_ips": {
            "terms": {
                "field": "src_ip",
                "size": 10
            }
        },
        "top_dest_ips": {
            "terms": {
                "field": "dest_ip",
                "size": 10
            }
        },
        "top_signatures": {
            "terms": {
                "field": "alert.signature.keyword",
                "size": 10
            }
        }
    }
}

result = es.search(index=index_name, body=aggs_query)

print("📊 Statistiques sur les alertes\n")
print("=" * 50)

# Alertes par catégorie
if 'alertes_par_categorie' in result['aggregations']:
    print("\n📁 Alertes par catégorie:")
    for bucket in result['aggregations']['alertes_par_categorie']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']}")

# Alertes par sévérité
if 'alertes_par_severite' in result['aggregations']:
    print("\n⚠️  Alertes par sévérité:")
    for bucket in result['aggregations']['alertes_par_severite']['buckets']:
        print(f"   Sévérité {bucket['key']}: {bucket['doc_count']}")

# Top IPs sources
if 'top_source_ips' in result['aggregations']:
    print("\n🌐 Top 10 IPs sources:")
    for bucket in result['aggregations']['top_source_ips']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']} événements")

# Top signatures
if 'top_signatures' in result['aggregations']:
    print("\n🔍 Top 10 signatures:")
    for bucket in result['aggregations']['top_signatures']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']}")

/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


📊 Statistiques sur les alertes


📁 Alertes par catégorie:
   A Network Trojan was detected: 1
   Attempted Information Leak: 1
   Potentially Bad Traffic: 1

⚠️  Alertes par sévérité:
   Sévérité 2: 2
   Sévérité 1: 1

🌐 Top 10 IPs sources:
   192.168.1.100: 3 événements
   198.51.100.10: 1 événements
   203.0.113.45: 1 événements

🔍 Top 10 signatures:
   ET MALWARE Known Malware IP: 1
   ET POLICY Suspicious inbound to mySQL port 3306: 1
   ET SCAN Potential SSH Scan: 1


## Partie 4 : Visualisation dans Kibana

### 4.1 Accès à Kibana

1. Ouvrez votre navigateur et allez sur `http://localhost:5601`
2. Connectez-vous avec les identifiants définis dans votre fichier `.env` (si sécurité activée)
3. Si c'est la première fois, vous devrez peut-être accepter les termes de licence

### 4.2 Création d'un index pattern

Pour visualiser les données dans Kibana, vous devez créer un **index pattern** :

1. Allez dans **Management** → **Stack Management** → **Index Patterns**
2. Cliquez sur **Create index pattern**
3. Entrez le pattern : `suricata-logs` ou `suricata-*` (si vous utilisez des dates)
4. Sélectionnez le champ de temps : `@timestamp` ou `timestamp`
5. Cliquez sur **Create index pattern**

### 4.3 Création de visualisations

Dans Kibana, créez les visualisations suivantes :

#### Visualisation 1 : Timeline des alertes
- **Type** : Line chart ou Area chart
- **Métrique** : Count
- **Buckets** : Date Histogram sur `@timestamp`
- **Filtre** : `event_type: alert`

#### Visualisation 2 : Top 10 signatures d'alertes
- **Type** : Horizontal Bar chart
- **Métrique** : Count
- **Buckets** : Terms sur `alert.signature.keyword` (Top 10)

#### Visualisation 3 : Répartition par sévérité
- **Type** : Pie chart
- **Métrique** : Count
- **Buckets** : Terms sur `alert.severity`

#### Visualisation 4 : Carte des IPs sources
- **Type** : Coordinate Map ou Tile Map
- **Métrique** : Count
- **Buckets** : Geohash sur `src_ip` (si géolocalisation activée)

### 4.4 Création d'un Dashboard

1. Allez dans **Dashboard** → **Create dashboard**
2. Ajoutez les visualisations créées précédemment
3. Configurez les filtres temporels
4. Sauvegardez le dashboard

**Note :** Pour ce TD, nous allons également créer des visualisations programmatiquement via l'API.

In [12]:
# Vérification de l'accès à Kibana (via l'API REST)
import requests
import urllib3

# Désactiver les avertissements SSL pour le développement
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

KIBANA_URL = "http://localhost:5601"
KIBANA_USER = "elastic"  # Modifiez si nécessaire
KIBANA_PASSWORD = None  # Définissez votre mot de passe si sécurité activée

# Test de connexion à Kibana
try:
    if KIBANA_PASSWORD:
        response = requests.get(
            f"{KIBANA_URL}/api/status",
            auth=(KIBANA_USER, KIBANA_PASSWORD),
            verify=False,
            timeout=5
        )
    else:
        response = requests.get(f"{KIBANA_URL}/api/status", timeout=5)
    
    if response.status_code == 200:
        print("✅ Kibana est accessible")
        status = response.json()
        print(f"   Version: {status.get('version', {}).get('number', 'N/A')}")
    else:
        print(f"⚠️  Kibana répond mais avec le code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Impossible de se connecter à Kibana")
    print("   Vérifiez que Kibana est démarré sur http://localhost:5601")
except Exception as e:
    print(f"⚠️  Erreur: {e}")

✅ Kibana est accessible
   Version: N/A


## Partie 5 : Alerting dans Kibana

### 5.1 Création de règles d'alerte

Kibana permet de créer des règles d'alerte basées sur des requêtes Elasticsearch. Voici quelques exemples de règles utiles pour un SIEM :

#### Règle 1 : Alerte sur haute sévérité
- **Condition** : Nombre d'alertes avec `severity <= 1` > 0
- **Action** : Envoyer une notification (email, Slack, etc.)

#### Règle 2 : Alerte sur scan de ports
- **Condition** : Plus de 10 alertes de type "SCAN" en 5 minutes
- **Action** : Notification immédiate

#### Règle 3 : Alerte sur IP suspecte
- **Condition** : Plus de 5 alertes depuis la même IP source en 10 minutes
- **Action** : Bloquer l'IP (si intégration avec firewall)

### 5.2 Configuration des alertes via l'API

Pour créer des alertes programmatiquement, utilisez l'API Kibana Alerting.

In [13]:
# Exemple de création d'une règle d'alerte Kibana
# Note: Cette fonctionnalité nécessite Kibana avec les fonctionnalités d'alerting activées

def create_alert_rule(rule_name, query, threshold, time_window="5m"):
    """
    Crée une règle d'alerte Kibana
    
    Args:
        rule_name: Nom de la règle
        query: Requête Elasticsearch (dict)
        threshold: Seuil d'alerte
        time_window: Fenêtre temporelle (ex: "5m", "1h")
    """
    alert_rule = {
        "name": rule_name,
        "consumer": "alerts",
        "enabled": True,
        "rule_type_id": ".es-query",
        "schedule": {
            "interval": "1m"
        },
        "actions": [
            {
                "group": "query matched",
                "id": "my-connector-id",  # À remplacer par un ID de connecteur réel
                "params": {
                    "message": f"Alerte: {rule_name} - Seuil dépassé!"
                }
            }
        ],
        "params": {
            "index": [index_name],
            "query": query,
            "size": 100,
            "timeField": "@timestamp",
            "timeWindowSize": time_window,
            "threshold": [
                {
                    "field": "count",
                    "value": threshold,
                    "comparator": ">"
                }
            ]
        }
    }
    
    return alert_rule

# Exemple: Règle d'alerte pour haute sévérité
high_severity_query = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}},
            {"range": {"alert.severity": {"lte": 1}}}
        ]
    }
}

alert_rule = create_alert_rule(
    rule_name="Alerte haute sévérité Suricata",
    query=high_severity_query,
    threshold=0,
    time_window="5m"
)

print("📋 Exemple de règle d'alerte créée:")
print(json.dumps(alert_rule, indent=2))

print("\n💡 Pour créer cette règle dans Kibana:")
print("   1. Allez dans Stack Management → Rules and Connectors")
print("   2. Créez un connecteur (email, Slack, etc.)")
print("   3. Créez une nouvelle règle avec les paramètres ci-dessus")

📋 Exemple de règle d'alerte créée:
{
  "name": "Alerte haute s\u00e9v\u00e9rit\u00e9 Suricata",
  "consumer": "alerts",
  "enabled": true,
  "rule_type_id": ".es-query",
  "schedule": {
    "interval": "1m"
  },
  "actions": [
    {
      "group": "query matched",
      "id": "my-connector-id",
      "params": {
        "message": "Alerte: Alerte haute s\u00e9v\u00e9rit\u00e9 Suricata - Seuil d\u00e9pass\u00e9!"
      }
    }
  ],
  "params": {
    "index": [
      "suricata-logs"
    ],
    "query": {
      "bool": {
        "must": [
          {
            "term": {
              "event_type": "alert"
            }
          },
          {
            "range": {
              "alert.severity": {
                "lte": 1
              }
            }
          }
        ]
      }
    },
    "size": 100,
    "timeField": "@timestamp",
    "timeWindowSize": "5m",
    "threshold": [
      {
        "field": "count",
        "value": 0,
        "comparator": ">"
      }
    ]
  }
}

💡 Po

In [14]:
# Détection de patterns suspects

# 1. Détection d'IP source avec beaucoup d'alertes (possible scan/attaque)
def detect_suspicious_source_ips(min_alerts=3):
    """Détecte les IPs sources avec un nombre élevé d'alertes"""
    query = {
        "size": 0,
        "query": {
            "term": {"event_type": "alert"}
        },
        "aggs": {
            "suspicious_ips": {
                "terms": {
                    "field": "src_ip",
                    "size": 20,
                    "min_doc_count": min_alerts
                },
                "aggs": {
                    "alert_details": {
                        "top_hits": {
                            "size": 5,
                            "_source": {
                                "includes": ["alert.signature", "alert.severity", "dest_ip", "dest_port"]
                            }
                        }
                    }
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print(f"🔍 IPs sources suspectes (≥{min_alerts} alertes):\n")
    for bucket in result['aggregations']['suspicious_ips']['buckets']:
        ip = bucket['key']
        count = bucket['doc_count']
        print(f"⚠️  {ip}: {count} alertes")
        
        # Afficher les détails des alertes
        for hit in bucket['alert_details']['hits']['hits']:
            source = hit['_source']
            alert = source.get('alert', {})
            print(f"   - {alert.get('signature', 'N/A')} (Sévérité: {alert.get('severity', 'N/A')})")
        print()
    
    return result

# 2. Détection d'alertes récentes (dernières 30 minutes)
def detect_recent_alerts(minutes=30):
    """Détecte les alertes récentes"""
    time_threshold = datetime.now() - timedelta(minutes=minutes)
    
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}}
            ],
            "filter": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": time_threshold.isoformat()
                        }
                    }
                }
            ]
        }
    }
    
    result = es.search(index=index_name, query=query, size=100, sort=[{"@timestamp": {"order": "desc"}}])
    
    print(f"🕐 Alertes des dernières {minutes} minutes: {result['hits']['total']['value']}\n")
    
    for hit in result['hits']['hits']:
        source = hit['_source']
        alert = source.get('alert', {})
        timestamp = source.get('@timestamp', source.get('timestamp', 'N/A'))
        print(f"⏰ {timestamp}")
        print(f"   {alert.get('signature', 'N/A')}")
        print(f"   {source.get('src_ip', 'N/A')} → {source.get('dest_ip', 'N/A')}")
        print()
    
    return result

# Exécuter les détections
print("=" * 60)
detect_suspicious_source_ips(min_alerts=1)
print("\n" + "=" * 60)
detect_recent_alerts(minutes=60)

🔍 IPs sources suspectes (≥1 alertes):

⚠️  192.168.1.100: 1 alertes
   - ET POLICY Suspicious inbound to mySQL port 3306 (Sévérité: 2)

⚠️  198.51.100.10: 1 alertes
   - ET MALWARE Known Malware IP (Sévérité: 1)

⚠️  203.0.113.45: 1 alertes
   - ET SCAN Potential SSH Scan (Sévérité: 2)


🕐 Alertes des dernières 60 minutes: 3

⏰ 2026-01-17T15:32:40.125407
   ET POLICY Suspicious inbound to mySQL port 3306
   192.168.1.100 → 10.0.0.50

⏰ 2026-01-17T15:27:40.125427
   ET SCAN Potential SSH Scan
   203.0.113.45 → 10.0.0.50

⏰ 2026-01-17T15:12:40.125446
   ET MALWARE Known Malware IP
   198.51.100.10 → 10.0.0.50



ObjectApiResponse({'took': 10, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': None, 'hits': [{'_index': 'suricata-logs', '_id': '1', '_score': None, '_source': {'timestamp': '2026-01-17T15:32:40.125407', 'event_type': 'alert', 'src_ip': '192.168.1.100', 'src_port': 54321, 'dest_ip': '10.0.0.50', 'dest_port': 80, 'proto': 'TCP', 'alert': {'action': 'allowed', 'gid': 1, 'signature_id': 2012896, 'rev': 1, 'signature': 'ET POLICY Suspicious inbound to mySQL port 3306', 'category': 'Potentially Bad Traffic', 'severity': 2}, 'flow_id': 1234567890, 'in_iface': 'eth0', '@timestamp': '2026-01-17T15:32:40.125407'}, 'sort': [1768663960125]}, {'_index': 'suricata-logs', '_id': '2', '_score': None, '_source': {'timestamp': '2026-01-17T15:27:40.125427', 'event_type': 'alert', 'src_ip': '203.0.113.45', 'src_port': 44321, 'dest_ip': '10.0.0.50', 'dest_port': 22, 'proto': 'TCP', 'alert': {'action': 

In [15]:
# Exemple de requête pour analyser les patterns temporels (utile pour ML)
# Cette requête peut être utilisée comme base pour un job ML

def analyze_temporal_patterns():
    """Analyse les patterns temporels dans les alertes"""
    
    # Agrégation par heure de la journée
    query = {
        "size": 0,
        "query": {
            "term": {"event_type": "alert"}
        },
        "aggs": {
            "alerts_by_hour": {
                "date_histogram": {
                    "field": "@timestamp",
                    "calendar_interval": "hour",
                    "min_doc_count": 1
                },
                "aggs": {
                    "by_severity": {
                        "terms": {
                            "field": "alert.severity",
                            "size": 5
                        }
                    },
                    "unique_ips": {
                        "cardinality": {
                            "field": "src_ip"
                        }
                    }
                }
            },
            "alerts_by_day": {
                "date_histogram": {
                    "field": "@timestamp",
                    "calendar_interval": "day",
                    "min_doc_count": 1
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print("📈 Analyse des patterns temporels\n")
    
    # Afficher les alertes par heure
    if 'alerts_by_hour' in result['aggregations']:
        print("🕐 Distribution des alertes par heure:")
        for bucket in result['aggregations']['alerts_by_hour']['buckets']:
            time_str = bucket['key_as_string']
            count = bucket['doc_count']
            unique_ips = bucket['unique_ips']['value']
            print(f"   {time_str}: {count} alertes ({unique_ips} IPs uniques)")
    
    print(f"\n📊 Total d'alertes analysées: {result['hits']['total']['value']}")
    
    return result

# Exécuter l'analyse
try:
    analyze_temporal_patterns()
except Exception as e:
    print(f"⚠️  Erreur lors de l'analyse: {e}")
    print("   Assurez-vous que les données contiennent des timestamps valides")

📈 Analyse des patterns temporels

🕐 Distribution des alertes par heure:
   2026-01-17T15:00:00.000Z: 3 alertes (3 IPs uniques)

📊 Total d'alertes analysées: 3


### 6.4 Détection d'anomalies basique (sans ML)

Même sans activer ML, nous pouvons créer des règles de détection d'anomalies basiques.

In [16]:
# Détection d'anomalies basique : identifier les comportements inhabituels

def detect_anomalies():
    """Détecte des anomalies basiques dans les logs"""
    
    anomalies = []
    
    # 1. IP source avec trop d'alertes différentes (possible botnet/scan)
    query1 = {
        "size": 0,
        "query": {"term": {"event_type": "alert"}},
        "aggs": {
            "ips_with_many_alerts": {
                "terms": {
                    "field": "src_ip",
                    "size": 10
                },
                "aggs": {
                    "unique_signatures": {
                        "cardinality": {
                            "field": "alert.signature_id"
                        }
                    },
                    "unique_dest_ips": {
                        "cardinality": {
                            "field": "dest_ip"
                        }
                    }
                }
            }
        }
    }
    
    result1 = es.search(index=index_name, body=query1)
    
    print("🔍 Détection d'anomalies basiques\n")
    print("=" * 60)
    
    # Analyser les résultats
    for bucket in result1['aggregations']['ips_with_many_alerts']['buckets']:
        ip = bucket['key']
        total_alerts = bucket['doc_count']
        unique_signatures = bucket['unique_signatures']['value']
        unique_dest_ips = bucket['unique_dest_ips']['value']
        
        # Critères d'anomalie
        if unique_signatures > 3 or unique_dest_ips > 5:
            anomaly = {
                "type": "Suspicious scanning activity",
                "ip": ip,
                "total_alerts": total_alerts,
                "unique_signatures": unique_signatures,
                "unique_dest_ips": unique_dest_ips
            }
            anomalies.append(anomaly)
            
            print(f"🚨 ANOMALIE DÉTECTÉE:")
            print(f"   IP: {ip}")
            print(f"   Type: Activité de scan suspecte")
            print(f"   Raison: {unique_signatures} signatures différentes, {unique_dest_ips} IPs de destination différentes")
            print(f"   Total alertes: {total_alerts}")
            print()
    
    # 2. Alertes de haute sévérité récentes
    query2 = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}},
                {"range": {"alert.severity": {"lte": 1}}}
            ],
            "filter": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": (datetime.now() - timedelta(hours=1)).isoformat()
                        }
                    }
                }
            ]
        }
    }
    
    result2 = es.search(index=index_name, query=query2, size=100)
    
    if result2['hits']['total']['value'] > 0:
        print("🚨 ALERTES CRITIQUES RÉCENTES:")
        for hit in result2['hits']['hits']:
            source = hit['_source']
            alert = source.get('alert', {})
            print(f"   - {alert.get('signature', 'N/A')}")
            print(f"     Source: {source.get('src_ip', 'N/A')}")
            print(f"     Sévérité: {alert.get('severity', 'N/A')}")
        print()
    
    return anomalies

# Exécuter la détection
anomalies = detect_anomalies()

if not anomalies:
    print("✅ Aucune anomalie majeure détectée avec les critères actuels")

🔍 Détection d'anomalies basiques



🚨 ALERTES CRITIQUES RÉCENTES:
   - ET MALWARE Known Malware IP
     Source: 198.51.100.10
     Sévérité: 1

✅ Aucune anomalie majeure détectée avec les critères actuels


## Partie 7 : Exercices pratiques

### Exercice 1 : Recherche d'IPs malveillantes

Créez une requête pour trouver toutes les alertes provenant d'une IP spécifique et analysez les signatures déclenchées.

### Exercice 2 : Analyse de trafic HTTP

Recherchez tous les événements HTTP et identifiez les requêtes suspectes (ex: tentatives d'accès à `/admin`, `/wp-admin`, etc.).

### Exercice 3 : Dashboard personnalisé

Créez un dashboard Kibana avec :
- Timeline des alertes
- Top 10 des signatures
- Carte des IPs sources
- Graphique de sévérité

### Exercice 4 : Règle d'alerte personnalisée

Créez une règle d'alerte qui se déclenche lorsqu'une IP génère plus de 5 alertes en 10 minutes.

### Exercice 5 : Détection de patterns

Écrivez une requête pour détecter les scans de ports (plusieurs tentatives de connexion sur différents ports depuis la même IP).

In [17]:
# EXERCICE 1 : Recherche d'IPs malveillantes
# Complétez cette fonction pour rechercher toutes les alertes d'une IP spécifique

def search_alerts_by_ip(ip_address):
    """
    Recherche toutes les alertes pour une IP source donnée
    
    Args:
        ip_address: Adresse IP à rechercher (ex: "192.168.1.100")
    
    Returns:
        Résultats de la recherche
    """
    # TODO: Créer la requête Elasticsearch
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}},
                {"term": {"src_ip": ip_address}}
            ]
        }
    }
    
    result = es.search(index=index_name, query=query, size=100)
    
    print(f"🔍 Alertes pour l'IP {ip_address}: {result['hits']['total']['value']}\n")
    
    signatures = {}
    for hit in result['hits']['hits']:
        source = hit['_source']
        alert = source.get('alert', {})
        sig_id = alert.get('signature_id', 'N/A')
        sig_name = alert.get('signature', 'N/A')
        
        if sig_id not in signatures:
            signatures[sig_id] = {
                "name": sig_name,
                "count": 0,
                "severity": alert.get('severity', 'N/A')
            }
        signatures[sig_id]["count"] += 1
    
    print("📋 Signatures déclenchées:")
    for sig_id, info in signatures.items():
        print(f"   [{sig_id}] {info['name']}: {info['count']} fois (Sévérité: {info['severity']})")
    
    return result

# Test avec une IP de test
# Récupérer d'abord quelques alertes pour obtenir une IP de test
test_query = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}}
        ]
    }
}
test_result = es.search(index=index_name, query=test_query, size=1)

if test_result['hits']['total']['value'] > 0:
    test_ip = test_result['hits']['hits'][0]['_source'].get('src_ip')
    if test_ip:
        print(f"\n🧪 Test avec l'IP: {test_ip}\n")
        search_alerts_by_ip(test_ip)
    else:
        print("⚠️  Aucune IP source trouvée dans les alertes")
else:
    print("⚠️  Aucune alerte trouvée pour le test")


🧪 Test avec l'IP: 192.168.1.100

🔍 Alertes pour l'IP 192.168.1.100: 1

📋 Signatures déclenchées:
   [2012896] ET POLICY Suspicious inbound to mySQL port 3306: 1 fois (Sévérité: 2)


In [18]:
# EXERCICE 2 : Analyse de trafic HTTP suspect
# Recherchez les requêtes HTTP suspectes

def find_suspicious_http_requests():
    """Recherche les requêtes HTTP suspectes"""
    
    # URLs suspectes communes
    suspicious_paths = ["/admin", "/wp-admin", "/phpmyadmin", "/.env", "/config.php"]
    
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "http"}}
            ],
            "should": [
                {"wildcard": {"http.url": f"*{path}*"}} for path in suspicious_paths
            ],
            "minimum_should_match": 1
        }
    }
    
    result = es.search(index=index_name, query=query, size=100)
    
    print(f"🚨 Requêtes HTTP suspectes trouvées: {result['hits']['total']['value']}\n")
    
    for hit in result['hits']['hits']:
        source = hit['_source']
        http = source.get('http', {})
        print(f"⚠️  {source.get('src_ip', 'N/A')} → {http.get('hostname', 'N/A')}{http.get('url', 'N/A')}")
        print(f"   Méthode: {http.get('http_method', 'N/A')}, Status: {http.get('status', 'N/A')}")
        print()
    
    return result

# Exécuter la recherche
try:
    find_suspicious_http_requests()
except Exception as e:
    print(f"⚠️  Erreur: {e}")
    print("   Vérifiez que vous avez des événements HTTP dans vos logs")

🚨 Requêtes HTTP suspectes trouvées: 0



## Partie 8 : Résumé et bonnes pratiques

### Bonnes pratiques pour un SIEM

1. **Indexation optimale**
   - Utilisez des index avec rotation temporelle (ex: `suricata-YYYY.MM.DD`)
   - Configurez des index templates pour automatiser la création
   - Définissez des politiques de rétention (ILM - Index Lifecycle Management)

2. **Mapping des champs**
   - Définissez correctement les types de champs (IP, date, keyword, text)
   - Utilisez des champs `keyword` pour les agrégations
   - Utilisez des champs `text` pour la recherche full-text

3. **Sécurité**
   - Activez la sécurité Elasticsearch en production
   - Utilisez des certificats SSL/TLS
   - Configurez des rôles et permissions appropriés

4. **Performance**
   - Monitorer la santé du cluster
   - Ajuster le nombre de shards selon le volume de données
   - Utiliser des alias pour les index

5. **Alerting**
   - Créez des règles d'alerte pertinentes (pas trop, pas trop peu)
   - Testez régulièrement les alertes
   - Documentez les procédures de réponse aux incidents

6. **Maintenance**
   - Surveillez les logs Elasticsearch et Kibana
   - Effectuez des sauvegardes régulières
   - Mettez à jour régulièrement les règles Suricata

### Ressources supplémentaires

- **Documentation Suricata** : https://suricata.readthedocs.io/
- **Documentation Filebeat** : https://www.elastic.co/guide/en/beats/filebeat/current/index.html
- **Documentation Elasticsearch** : https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- **Documentation Kibana** : https://www.elastic.co/guide/en/kibana/current/index.html
- **Elastic Security** : https://www.elastic.co/security

### Prochaines étapes

1. Intégrer d'autres sources de logs (Apache, Nginx, Windows Event Logs, etc.)
2. Configurer des enrichissements (géolocalisation IP, threat intelligence)
3. Créer des playbooks de réponse aux incidents
4. Mettre en place une stack complète avec Logstash si nécessaire
5. Explorer Elastic Security (anciennement SIEM) pour des fonctionnalités avancées

In [19]:
# Fonction utilitaire : Nettoyage des ressources
def cleanup():
    """Supprime l'index de test (optionnel)"""
    response = input("Voulez-vous supprimer l'index 'suricata-logs'? (oui/non): ")
    if response.lower() in ['oui', 'o', 'yes', 'y']:
        if es.indices.exists(index=index_name):
            es.indices.delete(index=index_name)
            print(f"✅ Index '{index_name}' supprimé")
        else:
            print(f"ℹ️  L'index '{index_name}' n'existe pas")
    else:
        print("ℹ️  Index conservé")

# Décommenter pour nettoyer
# cleanup()

print("\n" + "=" * 60)
print("✅ TD terminé!")
print("=" * 60)
print("\n📚 N'oubliez pas de:")
print("   1. Explorer Kibana pour créer vos visualisations")
print("   2. Configurer des alertes dans Kibana")
print("   3. Tester avec de vrais logs Suricata")
print("   4. Documenter vos règles et procédures")
print("\n🎓 Bonne continuation avec votre SIEM!")


✅ TD terminé!

📚 N'oubliez pas de:
   1. Explorer Kibana pour créer vos visualisations
   2. Configurer des alertes dans Kibana
   3. Tester avec de vrais logs Suricata
   4. Documenter vos règles et procédures

🎓 Bonne continuation avec votre SIEM!
